In [1]:
# Default imports provided by IBM's notebook, which is where I created the code.
import numpy as np

# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, transpile, Aer, IBMQ
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from ibm_quantum_widgets import *
from qiskit.providers.aer import QasmSimulator

# These are some items I imported individually, just to make sure I had everything I needed before starting the development my program.
from qiskit import ClassicalRegister, QuantumRegister
from qiskit import QuantumCircuit
from qiskit import *

# Loading your IBM Quantum account(s)
provider = IBMQ.load_account()

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [2]:
def addition_function(first, second): # Creating an addition function with all the quantum-related parts before using a for loop to create a multiplication function. 
    # Converting the user-inputted numbers into binary numbers
    first = bin(first).replace("0b", "")
    second = bin(second).replace("0b", "")

    # Need the length of the two numbers to set up quantum and classical registers with suitable lengths.
    len_1 = len(first)
    len_2 = len(second)
    n = len_1+len_2-1

    # num_2 has an extra qubit for the sum.
    #The classical register has n+1 bits, allowing for the sum to be read.

    num_1 = QuantumRegister(n) # Needed for the first number.
    num_2 = QuantumRegister(n+1) # Needed for the second number and eventually, the sum of the two numbers.
    carry = QuantumRegister(n) # Needed for any carrying
    c = ClassicalRegister(n+1) 

    #Combining the previously defined registers into a single circuit.
    circ = QuantumCircuit(num_1, num_2, carry, c)

    #Setting up the registers based upon the lengths of the two binary numbers.
    for i in range(len_1):
        if first[i] == "1": 
            circ.x(num_1[len_1 - (i+1)]) # Flips the qubit from zero the one, i.e. the x-gate
    for i in range(len_2):
       if second[i] == "1":
          circ.x(num_2[len_2 - (i+1)]) # Flips the qubit from zero the one, i.e. the x-gate

    # Implementing a carry (ccx or Toffoli) gate that is applied on (num_1[i], num_2[i], and carry[i]) with the output put to carry[i+1]
    for i in range(n-1):
        circ.ccx(num_1[i], num_2[i], carry[i+1])
        circ.cx(num_1[i], num_2[i])
        circ.ccx(carry[i], num_2[i], carry[i+1])


    # For the final iteration of the Toffoli gate, instead of putting the result to carry[n], we use num_2[n], which is why carry has only n bits, with carry[n-1] being the last carry bit.
    # Also, we pre-defined num_2 having a slot for the sum of the two numbers.
    circ.ccx(num_1[n-1], num_2[n-1], num_2[n])
    circ.cx(num_1[n-1], num_2[n-1])
    circ.ccx(carry[n-1], num_2[n-1], num_2[n])

    # Reversing the gate operation performed on num_2[n-1]
    circ.cx(carry[n-1], num_2[n-1])
    # Reversing the gate operations performed during the carry gate implementations. This is done to make sure the sum gates are given the correct input bit states

    for i in range(n-1):
        # Note the usage of the controlled-not gate and the Toffoli gate for the reversal procedure.
        circ.ccx(carry[(n-2)-i], num_2[(n-2)-i], carry[(n-1)-i])
        circ.cx(num_1[(n-2)-i], num_2[(n-2)-i])
        circ.ccx(num_1[(n-2)-i], num_2[(n-2)-i], carry[(n-1)-i])
        # These two operations act as a sum gate. If a control bit is at the ket one state, then the target bit num_2[(n-2)-i] is flipped (controlled gate, in essence).
        circ.cx(carry[(n-2)-i], num_2[(n-2)-i])
        circ.cx(num_1[(n-2)-i], num_2[(n-2)-i])


    # Measuring all qubits and storing them into the previously created classical register.
    for i in range(n+1):
        circ.measure(num_2[i], c[i])

    # Setting up and running my program onto the idealized Statevector-Simulator for optimial results, due to the small input size.
    backend = Aer.get_backend("statevector_simulator")
    job = execute(circ, backend)
    counts = job.result().get_counts()

    return (int(next(iter(counts)), 2)) # Returning the final integer

    # print(list(counts.keys())[0]) - useful for displaying the output as a binary number, but not needed

In [9]:
def multiplier(first, second): # This is the function that uses a for loop to do multiplication with the previously created addition_function.
    product = 0
    for i in range (0, second):
        sum = addition_function(first,first)
        product += sum
    product /= 2    
    return product    

# This is just some test code, and can be changed if needed
test = multiplier(5,6)
print(test)

30.0
